In [1]:
from utils import *
df_tracks = read_avro('data/tracks.avro')

,id,timestamp,action,track_id,user_id
0,0,2024-01-01T06:29:35,LIKE,1O5SLSkcxOGeA7bC0YySGZ,0
1,0,2024-01-01T20:48:23,PAUSE,3gLHdiOYsQPQ21LNnDsRNX,0
2,0,2024-01-01T21:17:25,QUIT,2EVQpdgf4JBNxpCEpkx26S,0
3,0,2024-01-01T20:16:18,ADD_TO_PLAYLIST,4mHeRG0dHruaY2kAIsg1Od,0
4,0,2024-01-01T04:04:45,PAUSE,1KQr7XLIu4sCMtWSjEGljb,0
...,...,...,...,...,...
392539,0,2024-02-07T19:49:14,PAUSE,6iQS9pIrx5Zkid4L145zkR,99
392540,0,2024-02-07T15:12:24,LIKE,5HIr9EyCMr8Gwqcd1rYgcq,99
392541,0,2024-02-07T01:58:35,DOWNLOAD,065zGV3F2qhNRPtMydcit4,99
392542,0,2024-02-07T06:31:52,DOWNLOAD,3x55W2TPzlrxNssAyhD1hG,99


In [3]:
df_artists = pd.read_csv(
    '/Users/yassine/Desktop/IE/4th year/2nd sem/stream analytics/datasets/artists.csv')
df_artists

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0
...,...,...,...,...,...
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,['black comedy'],Ali Siddiq,34
1162091,6LogY6VMM3jgAE6fPzXeMl,46.0,[],Rodney Laney,2
1162092,19boQkDEIay9GaVAWkUhTa,257.0,[],Blake Wexler,10
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,['black comedy'],Donnell Rawlings,15


In [21]:
from datetime import datetime
import pandas as pd

# Assuming df_tracks is your DataFrame with a 'release_date' column
# Convert 'release_date' to datetime objects with errors='coerce'
df_tracks['release_date'] = pd.to_datetime(
    df_tracks['release_date'], errors='coerce')

# Handle cases where 'release_date' has only the year or year-month information
df_tracks['release_date'] = df_tracks['release_date'].apply(
    lambda x: x.replace(day=1, month=1) if pd.notnull(x) and (x.year < 1900 or x.month == 1) else x)

# Filter out rows with NaT in 'release_date'
df_tracks = df_tracks.dropna(subset=['release_date'])

# Define the reference date
reference_date = datetime(2024, 1, 1)

# Calculate the time difference in months and calculate the newness score
df_tracks['newness_score'] = 1 / \
    (1 + (reference_date - df_tracks['release_date']).dt.days // 30)

# Alternatively, you can normalize the newness_score to a specific range if needed
# df_tracks['newness_score'] = df_tracks['newness_score'] / df_tracks['newness_score'].max()

# Print or use df_tracks[['release_date', 'newness_score']] as needed
print(df_tracks[['release_date', 'newness_score']])
print(len(df_tracks.release_date))

/var/folders/vk/7gng6fhd2fzb0wxymcgl6j880000gn/T/ipykernel_81963/3364372519.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tracks['release_date'] = pd.to_datetime(df_tracks['release_date'], errors='coerce')


       release_date  newness_score
0        1922-02-22       0.000806
1        1922-06-01       0.000808
2        1922-03-21       0.000806
3        1922-03-21       0.000806
9        1922-03-29       0.000807
...             ...            ...
586596   2020-09-26       0.025000
586597   2020-10-21       0.025641
586598   2020-09-02       0.024390
586599   2021-03-05       0.028571
586600   2015-07-01       0.009615

[448010 rows x 2 columns]
448010


/var/folders/vk/7gng6fhd2fzb0wxymcgl6j880000gn/T/ipykernel_81963/3364372519.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tracks['release_date'] = df_tracks['release_date'].apply(


In [4]:
# read json dictionary

import json
with open('/Users/yassine/Desktop/IE/4th year/2nd sem/stream analytics/datasets/dict_artists.json') as json_file:
    artist_dict = json.load(json_file)

In [72]:
df_copy = read_avro('data/tracks.avro')
print(len(df_copy))

586601


In [94]:
df_tracks = df_copy.copy()

In [95]:
import pandas as pd
import numpy as np

mask = df_tracks['release_date'].astype(str).apply(len) == 4

df_tracks.loc[mask, 'months_elapsed'] = (2024 - pd.to_numeric(
    df_tracks.loc[mask, 'release_date'])+1) * 12  # +1 to include the release year

reference_date = pd.to_datetime('2024-01-01')

try:
    months_elapsed, _ = divmod((reference_date - pd.to_datetime(
        df_tracks.loc[~mask, 'release_date'], errors='coerce')).dt.days, 30)
    df_tracks.loc[~mask, 'months_elapsed'] = months_elapsed.fillna(0)
except ValueError as e:
    print(f"Error: {e}")
    print("Problematic values:")
    print(df_tracks.loc[~mask, 'release_date'])
    df_tracks.loc[~mask, 'months_elapsed'] = np.nan

print(df_tracks[['release_date', 'months_elapsed']])

       release_date  months_elapsed
0        1922-02-22          1240.0
1        1922-06-01          1236.0
2        1922-03-21          1239.0
3        1922-03-21          1239.0
4              1922          1236.0
...             ...             ...
586596   2020-09-26            39.0
586597   2020-10-21            38.0
586598   2020-09-02            40.0
586599   2021-03-05            34.0
586600   2015-07-01           103.0

[586601 rows x 2 columns]


In [97]:
# Assuming df_tracks is your DataFrame
df_tracks['newness_score'] = 1 / df_tracks['months_elapsed']

# Print or use df_tracks[['release_date', 'newness_score']] as needed
print(df_tracks[['release_date', 'newness_score']])

       release_date  newness_score
0        1922-02-22       0.000806
1        1922-06-01       0.000809
2        1922-03-21       0.000807
3        1922-03-21       0.000807
4              1922       0.000809
...             ...            ...
586596   2020-09-26       0.025641
586597   2020-10-21       0.026316
586598   2020-09-02       0.025000
586599   2021-03-05       0.029412
586600   2015-07-01       0.009709

[586601 rows x 2 columns]


In [36]:
from datetime import datetime
import pandas as pd

# Assuming df_tracks is your DataFrame with a 'release_date' column
# Convert 'release_date' to datetime objects with errors='coerce'
df_tracks['release_date'] = pd.to_datetime(
    df_tracks['release_date'], errors='coerce')

# Handle cases where 'release_date' has only the year or year-month information
df_tracks['release_date'] = df_tracks['release_date'].apply(
    lambda x: x.replace(day=1, month=1) if pd.notnull(x) and (pd.isna(x.month) or x.month == 1) else x)

# Create a copy of the DataFrame to avoid the SettingWithCopyWarning
df_tracks_copy = df_tracks.copy()

# Filter out rows with NaT in 'release_date'
df_tracks_copy = df_tracks_copy.dropna(subset=['release_date'])

# Define the reference date
reference_date = datetime(2024, 1, 1)

# Calculate the time difference in months and calculate the newness score
df_tracks_copy['newness_score'] = 1 / \
    (1 + ((reference_date - df_tracks_copy['release_date']).dt.days // 30))

# Alternatively, you can normalize the newness_score to a specific range if needed
# df_tracks_copy['newness_score'] = df_tracks_copy['newness_score'] / df_tracks_copy['newness_score'].max()

# Print or use df_tracks_copy[['release_date', 'newness_score']] as needed
print(df_tracks_copy[['release_date', 'newness_score']])

       release_date  newness_score
0        1922-02-22       0.000806
1        1922-06-01       0.000808
2        1922-03-21       0.000806
3        1922-03-21       0.000806
9        1922-03-29       0.000807
...             ...            ...
586596   2020-09-26       0.025000
586597   2020-10-21       0.025641
586598   2020-09-02       0.024390
586599   2021-03-05       0.028571
586600   2015-07-01       0.009615

[448010 rows x 2 columns]


In [98]:
df_tracks.columns

Index(['track_id', 'duration', 'artist', 'name', 'popularity', 'release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'months_elapsed', 'newness_score'],
      dtype='object')

In [100]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming df_tracks is your DataFrame
# Extract relevant audio features for clustering
audio_features = df_tracks[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

# Standardize the data
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(audio_features)

# Determine the number of clusters (you may need to experiment with this)
num_clusters = 10  # Adjust this based on your preference

# Apply k-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_tracks['genre'] = kmeans.fit_predict(scaled_audio_features)

# Print or use df_tracks[['track_id', 'genre']] as needed
print(df_tracks[['track_id', 'genre']])

/Users/yassine/miniconda3/envs/stream_analytics_project/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                      track_id  genre
0       35iwgR4jXetI318WEWsa1Q      3
1       021ht4sdgPcrDgSk7JTbKY      7
2       07A5yehtSnoedViJAZkNnc      1
3       08FmqUhxtyLTn6pAh6bk45      6
4       08y9GfoqCWfOGsKdwojr5e      8
...                        ...    ...
586596  5rgu12WBIHQtvej2MdHSH0      8
586597  0NuWgxEp51CutD2pJoF4OM      5
586598  27Y1N4Q4U3EfDU5Ubw8ws2      8
586599  45XJsGpFTyzbzeWK8VzR8S      9
586600  5Ocn6dZ3BJFPWh4ylwFXtn      9

[586601 rows x 2 columns]


In [101]:
df_tracks.genre.unique()

array([3, 7, 1, 6, 8, 0, 5, 2, 9, 4], dtype=int32)

In [102]:
df_tracks.columns

'track_id', 'duration', 'artist', 'name', 'popularity', 'r ','newness_score', 'genre'],
      

Index(['track_id', 'duration', 'artist', 'name', 'popularity', 'release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'months_elapsed', 'newness_score', 'genre'],
      dtype='object')